In [202]:
import pandas as pd
import datetime as dt


path_const= r'C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\{}'
path_price_index= r'C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\warehouses.xlsx'

path_oil_brent= r'C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Oil\Brent Oil Futures Historical Data; monthly.csv'
path_TASI= r'C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Markets\Tadawul All Share Historical Data monthly.csv'
path_Gold= r'C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\gold\Gold Futures Historical Data monthly.csv'
path_interest_rate= r'C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Fed fund rates\FEDFUNDS.csv'


# Create date column for both dataframes:
# this function takes a string object which is in a date format and covert it to a date object
def str_to_datetime(s):
  split = s.split('/')
  month, day, year = int(split[0]), int(split[1]), int(split[2])
  return dt.datetime(year=year, month=month, day=day)

def str_to_datetime_ii(s):
  split = s.split('-')
  year, month, day = int(split[0]), int(split[1]), int(split[2])
  return dt.datetime(year=year, month=month, day=day)

In [206]:
#           oil
df_oil_brent= pd.read_csv(path_oil_brent)
df_oil_brent['Date'] = df_oil_brent['Date'].apply(str_to_datetime)
df_oil_brent.index = df_oil_brent.pop('Date')
# we keep only the needed columns
df_oil_brent= pd.DataFrame(df_oil_brent['Price'])
df_oil_brent.columns= ['oil_brent']

#           TASI
df_TASI= pd.read_csv(path_TASI)
df_TASI['Date'] = df_TASI['Date'].apply(str_to_datetime)
df_TASI.index = df_TASI.pop('Date')
# we keep only the needed columns
df_TASI= pd.DataFrame(df_TASI['Price'])
df_TASI.columns= ['tasi']

#           gold
df_gold= pd.read_csv(path_Gold)
df_gold['Date'] = df_gold['Date'].apply(str_to_datetime)
df_gold.index = df_gold.pop('Date')
# we keep only the needed columns
df_gold= pd.DataFrame(df_gold['Price'])
df_gold.columns= ['gold']

#           interest rates
df_interest_rates= pd.read_csv(path_interest_rate)
df_interest_rates['Date'] = df_interest_rates['DATE'].apply(str_to_datetime_ii)
df_interest_rates.index = df_interest_rates.pop('Date')
# we keep only the needed columns
df_interest_rates= pd.DataFrame(df_interest_rates['FEDFUNDS'])
df_interest_rates.columns= ['interest_rate']

In [207]:
df_const_mat= pd.read_excel(path_price_index)
df_const_mat.columns= ['cat.', 'type', 'price', 'month name', 'year', 'month', 'unit']
df_const_mat= df_const_mat.loc[:,['type', 'month','year', 'price']]
const_types= list(set(df_const_mat['type']))

dict_price_index= {}

for t in const_types:
    dict_price_index[t]= df_const_mat[df_const_mat['type'] == t]
    
for df_temp in dict_price_index:
    # prepare the TARGET
    print(filepath)
    dict_price_index[df_temp].pop('type')
    dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['month'].map(str)+ '/'  + '1' + '/' + dict_price_index[df_temp]['year'].map(str)
    dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['date'].apply(str_to_datetime)
    dict_price_index[df_temp].index = dict_price_index[df_temp].pop('date')
    dict_price_index[df_temp]= pd.DataFrame(dict_price_index[df_temp]['price'])

    # join data frames to prepare the PREDICTORS
    df_res = pd.merge(dict_price_index[df_temp], df_oil_brent, how='left', left_index=True, right_index=True)
    df_res = pd.merge(df_res,df_TASI, how='left', left_index=True, right_index=True)
    df_res = pd.merge(df_res,df_gold, how='left', left_index=True, right_index=True)
    df_res = pd.merge(df_res,df_interest_rates, how='left', left_index=True, right_index=True)
    
    # add historical data
    df_res['historical']= df_res['price'].shift(1)
    df_res['historical'].fillna(value=df_res['historical'].mean(), inplace=True)

    filename= df_temp+'.xlsx'
    filepath= path_const.format(filename)
    df_res.to_excel(filepath)#, index=False)

C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\خرسانة عادي 250ك.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\أسلاك كهربائية 2.5مم وطني.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\خشب تيك أفريقي.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\ألمنيوم (حلق جنب سعودي).xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\حديد تسليح 18مم وطني.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\خشب روماني.xlsx


C:\Users\Administrator\AppData\Local\Temp\ipykernel_26132\1391875489.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['month'].map(str)+ '/'  + '1' + '/' + dict_price_index[df_temp]['year'].map(str)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_26132\1391875489.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['date'].apply(str_to_datetime)
C:\Users\Administrator\AppData\L

C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\حديد تسليح 14مم وطني.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\بلوك اسود مقاس 20سم.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\كيابل كهربائية 50مم وطني.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\حديد تسليح 12مم وطني.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\بلاط كسر رخام بلدي.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\كيابل كهربائية 120مم وطني.xlsx


C:\Users\Administrator\AppData\Local\Temp\ipykernel_26132\1391875489.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['month'].map(str)+ '/'  + '1' + '/' + dict_price_index[df_temp]['year'].map(str)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_26132\1391875489.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['date'].apply(str_to_datetime)
C:\Users\Administrator\AppData\L

C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\حديد تسليح 6مم وطني.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\أسلاك كهربائية 4مم وطني.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\كيابل كهربائية 300مم وطني.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\إسمنت ابيض وطني.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\أسلاك كهربائية 6مم وطني.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\خرسانة مقاوم 350ك.xlsx


...

C:\Users\Administrator\AppData\Local\Temp\ipykernel_26132\1391875489.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['month'].map(str)+ '/'  + '1' + '/' + dict_price_index[df_temp]['year'].map(str)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_26132\1391875489.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['date'].apply(str_to_datetime)
C:\Users\Administrator\AppData\L

C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\رمل ناعم ابيض.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\جبس وطني.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\إسمنت أسود وطني.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\كيابل ربط الحديد صيني.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\حديد تسليح 10 مم وطني.xlsx
C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\خرسانة مقاوم 250ك.xlsx


C:\Users\Administrator\AppData\Local\Temp\ipykernel_26132\1391875489.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['month'].map(str)+ '/'  + '1' + '/' + dict_price_index[df_temp]['year'].map(str)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_26132\1391875489.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['date'].apply(str_to_datetime)
C:\Users\Administrator\AppData\L

C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\كيابل كهربائية 70مم وطني.xlsx


C:\Users\Administrator\AppData\Local\Temp\ipykernel_26132\1391875489.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['month'].map(str)+ '/'  + '1' + '/' + dict_price_index[df_temp]['year'].map(str)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_26132\1391875489.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dict_price_index[df_temp]['date'] = dict_price_index[df_temp]['date'].apply(str_to_datetime)


In [208]:
dataset_train= pd.read_excel(r'C:\Users\fnafisa\WORKSPACE\professional\Project PoC\Input\data\Construction materials\segregated\أسلاك كهربائية 2.5مم وطني.xlsx')
dataset_train.index= dataset_train.pop('date')
dataset_train= dataset_train.to_period(freq='m')
# select features (columns) to be involved into training and predictions
cols= list(dataset_train)[:]


# EXTRACT dates:
# store the date index in a list 
datelist_train= dataset_train.index.strftime('%Y-%m')
datelist_train= [dt.datetime.strptime(date, '%Y-%m').date() for date in datelist_train]


print('Training set shape == {}'.format(dataset_train.shape))
print('All timestamps == {}'.format(len(datelist_train)))
print('Featured selected: {}'.format(cols))
print('Sample of date list: {}...{}'.format(datelist_train[0:2], datelist_train[-3:-1]))

Training set shape == (117, 6)
All timestamps == 117
Featured selected: ['price', 'oil_brent', 'tasi', 'gold', 'interest_rate', 'historical']
Sample of date list: [datetime.date(2013, 1, 1), datetime.date(2013, 2, 1)]...[datetime.date(2022, 7, 1), datetime.date(2022, 8, 1)]


In [209]:
# select features (columns) to be involved into training and predictions
cols= list(dataset_train)[:]


# EXTRACT dates:
# store the date index in a list 
datelist_train= dataset_train.index.strftime('%Y-%m')
datelist_train= [dt.datetime.strptime(date, '%Y-%m').date() for date in datelist_train]


print('Training set shape == {}'.format(dataset_train.shape))
print('All timestamps == {}'.format(len(datelist_train)))
print('Featured selected: {}'.format(cols))
print('Sample of date list: {}...{}'.format(datelist_train[0:2], datelist_train[-3:-1]))

Training set shape == (117, 6)
All timestamps == 117
Featured selected: ['price', 'oil_brent', 'tasi', 'gold', 'interest_rate', 'historical']
Sample of date list: [datetime.date(2013, 1, 1), datetime.date(2013, 2, 1)]...[datetime.date(2022, 7, 1), datetime.date(2022, 8, 1)]


In [210]:
dataset_train

,price,oil_brent,tasi,gold,interest_rate,historical
date,,,,,,
2013-01,0.913,114.54,"7,043.55","1,662.00",0.14,0.819371
2013-02,0.911,110.42,"6,998.33","1,578.90",0.15,0.913000
2013-03,0.912,109.79,"7,125.73","1,595.70",0.14,0.911000
2013-04,0.908,102.04,"7,179.80","1,472.70",0.15,0.912000
2013-05,0.898,100.20,"7,404.12","1,393.00",0.11,0.908000
...,...,...,...,...,...,...
2022-05,1.180,115.60,"12,921.74","1,848.40",0.77,1.190000
2022-06,1.180,109.03,"11,523.25","1,812.00",1.21,1.180000
2022-07,1.180,103.97,"12,199.08","1,771.50",1.68,1.180000
